# OpenAlex Colombia

In [33]:
import requests
import time
import json
from IPython.display import JSON

In [ ]:
co=[]
i=0
count=0
baseurl='https://api.openalex.org/works?filter=authorships.institutions.country_code:co&mailto=restrepo@udea.edu.co&per-page=200'
nc='*'

In [113]:
while nc:
    i+=1
    url=baseurl+f'&cursor={nc}'
    try:
        r=requests.get(url)
        cotmp=r.json().get('results')
    except:
        print('ERROR: bad url')
        break
    if i==1:
        try:
            count=r.json().get('meta').get('count')
        except:
            print('ERROR: bad count')
            break
    kk=[d.pop('abstract_inverted_index') for d in cotmp if d.get('abstract_inverted_index')]
    try:
        nc=r.json().get('meta').get('next_cursor')
    except:
        print('ERROR: bad cursor')
        break
    #progress
    print(f'{i} → {200*i}/{count}',end='\r')
    #Append to dictionary list (WARNING: HIG RAM USAGE)
    for p in cotmp:
        co.append(p)
    #=== backup =====
    f=open('openalexco.jsonl','a')
    for p in cotmp:
        json.dump(p,f)
        f.write("\n")
    f.close()
    #API limits
    time.sleep(0.1)
    #EMERGENGY EXIT
    if i*200>count:
        break

In [114]:
f=open('openalexco.json','w')
json.dump(co,f)
f.close()


In [115]:
len(co)

343057

In [116]:
f=open('openalexco.jsonl','a')
for p in co:
    json.dump(p,f)
    f.write("\n")
f.close()

Medicine & Psychology Colombia

In [137]:
md=[d for d in co if len( {'Medicine','Psychology'}.intersection( 
       [ c.get('display_name') for c in d.get('concepts') if c.get('level')==0 ] 
          ) )>0 ]

In [155]:
s.add('B')
s

{'B'}

In [ ]:
insts=set()
for d in md:
    for a in d.get('authorships'):
        if a.get('institutions'):
            for i in a.get('institutions'):
                if i.get('country_code')=="CO":
                    insts.add(i.get('display_name'))

In [159]:
s

{'El Bosque University',
 'Fundación Santa Fe de Bogotá',
 'Fundación Universitaria de Ciencias de la Salud',
 'Fundación Valle del Lili',
 'Hospital Universitario de la Samaritana',
 'Icesi University',
 'Instituto Nacional de Salud',
 'National University of Colombia',
 'Pontificia Universidad Javeriana',
 'Universidad Autónoma de Occidente',
 'Universidad Santo Tomás',
 'Universidad Tecnológica de Bolívar',
 'Universidad del Rosario',
 'Universidad del Sinú',
 'University of Applied and Environmental Sciences',
 'University of Boyaca',
 'University of Cartagena',
 'University of the Coast'}

In [146]:
[[i.get('display_name') for i in a.get('institutions')] for a in d.get('authorships') if a.get('institutions')]

[['Universidad del Sinú']]

In [ ]:
[item for sublist in l for item in sublist]

In [142]:
JSON(d)

<IPython.core.display.JSON object>

In [119]:
JSON(co[0])

<IPython.core.display.JSON object>

In [72]:
cc=[]

In [73]:
list(range(2))

[0, 1]

In [74]:
for i in range(len(co)+1)
    for p in co[i]:
        cc.append(p)

In [97]:
co=cc

In [77]:
for p in cc[:10]:
    f=open('j.jsonl','a')
    json.dump(p,f)
    f.write("\n")

In [81]:
f=open('j.jsonl','r')

In [90]:
qq=[]
f=open('j.jsonl','r')
for d in f.readlines():
    qq.append(json.loads(d))

In [92]:
import pandas as pd

In [93]:
pd.DataFrame(qq)

,id,doi,title,display_name,publication_year,publication_date,ids,host_venue,type,open_access,...,concepts,mesh,alternate_host_venues,referenced_works,related_works,cited_by_api_url,counts_by_year,updated_date,created_date,abstract_inverted_index
0,https://openalex.org/W4220927483,https://doi.org/10.1590/1519-6984.254487,"New records of Coronatella (Crustacea, Branchi...","New records of Coronatella (Crustacea, Branchi...",2024,2024-01-01,{'openalex': 'https://openalex.org/W4220927483...,"{'id': 'https://openalex.org/V90590500', 'issn...",journal-article,"{'is_oa': True, 'oa_status': 'hybrid', 'oa_url...",...,"[{'id': 'https://openalex.org/C158710501', 'wi...","[{'descriptor_ui': 'D033361', 'descriptor_name...","[{'id': 'https://openalex.org/V90590500', 'dis...","[https://openalex.org/W74419506, https://opena...","[https://openalex.org/W4810741, https://openal...",https://api.openalex.org/works?filter=cites:W4...,[],2022-05-26,2022-04-03,NaN
1,https://openalex.org/W4283155905,https://doi.org/10.1590/1519-6984.259454,Ananas ananassoides (Baker) L.B.Sm. a bromelia...,Ananas ananassoides (Baker) L.B.Sm. a bromelia...,2024,2024-01-01,{'openalex': 'https://openalex.org/W4283155905...,"{'id': 'https://openalex.org/V90590500', 'issn...",journal-article,"{'is_oa': True, 'oa_status': 'hybrid', 'oa_url...",...,"[{'id': 'https://openalex.org/C86803240', 'wik...","[{'descriptor_ui': 'D031230', 'descriptor_name...","[{'id': 'https://openalex.org/V90590500', 'dis...","[https://openalex.org/W242768502, https://open...","[https://openalex.org/W595255, https://openale...",https://api.openalex.org/works?filter=cites:W4...,[],2022-07-28T22:44:38.042956,2022-06-21,NaN
2,https://openalex.org/W3173849640,https://doi.org/10.1590/1519-6984.247530,Intestinal protozoan infections and environmen...,Intestinal protozoan infections and environmen...,2023,2023-01-01,{'openalex': 'https://openalex.org/W3173849640...,"{'id': 'https://openalex.org/V90590500', 'issn...",journal-article,"{'is_oa': True, 'oa_status': 'hybrid', 'oa_url...",...,"[{'id': 'https://openalex.org/C86803240', 'wik...",[],"[{'id': 'https://openalex.org/V90590500', 'dis...","[https://openalex.org/W1985341132, https://ope...","[https://openalex.org/W20469624, https://opena...",https://api.openalex.org/works?filter=cites:W3...,[],2022-05-27,2021-07-05,NaN
3,https://openalex.org/W3200275451,https://doi.org/10.1590/1519-6984.250235,The first record of Bergamina lineolata (Chydo...,The first record of Bergamina lineolata (Chydo...,2023,2023-01-01,{'openalex': 'https://openalex.org/W3200275451...,"{'id': 'https://openalex.org/V90590500', 'issn...",journal-article,"{'is_oa': True, 'oa_status': 'hybrid', 'oa_url...",...,"[{'id': 'https://openalex.org/C86803240', 'wik...",[],"[{'id': 'https://openalex.org/V90590500', 'dis...","[https://openalex.org/W270376701, https://open...","[https://openalex.org/W24251540, https://opena...",https://api.openalex.org/works?filter=cites:W3...,"[{'year': 2022, 'cited_by_count': 1}]",2022-07-21T14:10:01.817273,2021-09-27,NaN
4,https://openalex.org/W4223556417,https://doi.org/10.1590/1806-9479.2021.256251,Implementación de pagos por servicios ambienta...,Implementación de pagos por servicios ambienta...,2023,2023-01-01,{'openalex': 'https://openalex.org/W4223556417...,"{'id': 'https://openalex.org/V121069939', 'iss...",journal-article,"{'is_oa': True, 'oa_status': 'hybrid', 'oa_url...",...,"[{'id': 'https://openalex.org/C15708023', 'wik...",[],"[{'id': 'https://openalex.org/V121069939', 'di...","[https://openalex.org/W267945720, https://open...","[https://openalex.org/W1004181, https://openal...",https://api.openalex.org/works?filter=cites:W4...,[],2022-05-27,2022-04-15,NaN
5,https://openalex.org/W4283699500,https://doi.org/10.1590/0103-8478cr20210703,Relationship of semen quality to inbreeding an...,Relationship of semen quality to inbreeding an...,2023,2023-01-01,{'openalex': 'https://openalex.org/W4283699500...,"{'id': 'https://openalex.org/V140190424', 'iss...",j

In [ ]:
for p in cc[:10]:
    f=open('j.jsonl','a')
    json.dump(p,f)
    f.write("\n")

In [79]:
import jsonlines

In [80]:
jsonlines.

Init signature:
jsonlines.Writer(
    fp: Union[IO[str], IO[bytes]] = None,
    *,
    compact: bool = False,
    sort_keys: bool = False,
    flush: bool = False,
    dumps: Callable[[Any], Union[str, bytes]] = <function default_dumps at 0x7fd9ac3ccee0>,
) -> None
Docstring:     
Writer for the jsonlines format.

Instances can be used as a context manager.

The `fp` argument must be a file-like object with a ``.write()``
method accepting either text (unicode) or bytes.

The `compact` argument can be used to to produce smaller output.

The `sort_keys` argument can be used to sort keys in json objects,
and will produce deterministic output.

For more control, provide a a custom encoder callable using the
`dumps` argument. The callable must produce (unicode) string output.
If specified, the `compact` and `sort` arguments will be ignored.

When the `flush` argument is set to ``True``, the writer will call
``fp.flush()`` after each written line.

:param fp: writable file-like object
:param

In [27]:
f=open('openalexco.json','r')
co=json.load(f)
f.close()

In [15]:
d={'A':1}

In [17]:
d.pop('A')

1

[{'Biological': [0],
  'samples': [1],
  'obtained': [2],
  'from': [3, 65],
  'a': [4, 216],
  'small': [5],
  'temporary': [6],
  'pond': [7],
  'of': [8,
   23,
   26,
   38,
   40,
   47,
   71,
   108,
   117,
   125,
   135,
   143,
   173,
   189,
   200,
   211,
   215,
   221],
  'northern': [9],
  'Colombia': [10],
  'yielded': [11],
  'the': [12, 24, 36, 58, 69, 72, 148, 153, 157, 187, 204],
  'first': [13],
  'record': [14],
  'Coronatella': [15, 41, 73],
  'undata': [16, 42, 61, 87],
  'Sousa,': [17, 163],
  'Elmoor-Loureiro': [18, 164],
  'and': [19, 22, 43, 45, 52, 156, 181],
  'Santos,': [20, 166],
  '2015': [21, 167],
  'male': [25, 46, 158],
  'C.': [27, 48, 60, 82, 86, 92, 145, 161],
  'monacantha': [28, 49, 74, 146],
  '(Sars,': [29, 84],
  '1901)': [30],
  'for': [31],
  'Colombia.': [32],
  'In': [33],
  'this': [34],
  'study,': [35],
  'morphology': [37],
  'female': [39, 44],
  'was': [50, 62],
  'described': [51, 64],
  'compared': [53],
  'to': [54, 77, 81, 1